In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adamax, Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.metrics import Mean
from tensorflow.keras.utils import to_categorical
from copy import deepcopy

In [2]:
import import_ipynb

import Model 
from Model import *

import ewc
from ewc import *

import utils
from utils import *

In [3]:
data = pd.read_pickle('mon_data.pkl')

In [4]:
# 모델 빌드 
model = DFNet.build(input_shape=(10000, 1), classes=95)

In [5]:
# Training the DF model
NB_EPOCH = 30   # Number of training epoch
BATCH_SIZE = 128 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 10000 # Packet sequence length
OPTIMIZER = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) # Optimizer

c:\ProgramData\Anaconda3\envs\nAIvis\lib\site-packages\keras\optimizers\optimizer_v2\adamax.py:99: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
# 모델 컴파일
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
print ("Model compiled")

Model compiled


In [7]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)  # 즉시 실행 모드 강제

first_part = split_by_label(data, 0, 49)
train, test = split_train_test(first_part, test_size=0.2, random_state=11)

# 3D ndarray 로 변환 
train_seq, train_label = split_data_label(train)

train_seq = np.stack(train_seq.values)
train_seq = train_seq[..., np.newaxis]

train_label = train_label.values
train_label = to_categorical(train_label, num_classes=95)
                

history = model.fit(x=train_seq, y=train_label, epochs=30, verbose=1)
optimal_weights = [w.numpy() for w in model.trainable_weights]
fisher_matrix = compute_fisher_matrix(model, train_seq, num_sample=1000)

print(f"Data_A training accuracy: {history.history['accuracy'][-1]:.4f}")

Epoch 1/30


c:\ProgramData\Anaconda3\envs\nAIvis\lib\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


250/250 [==============================] - 35s 101ms/step - loss: 3.9484 - accuracy: 0.0656
Epoch 2/30
250/250 [==============================] - 24s 98ms/step - loss: 3.3720 - accuracy: 0.1328
Epoch 3/30
250/250 [==============================] - 25s 100ms/step - loss: 2.9949 - accuracy: 0.2118
Epoch 4/30
250/250 [==============================] - 26s 103ms/step - loss: 2.7150 - accuracy: 0.2673
Epoch 5/30
250/250 [==============================] - 25s 102ms/step - loss: 2.4892 - accuracy: 0.3192
Epoch 6/30
250/250 [==============================] - 25s 102ms/step - loss: 2.3230 - accuracy: 0.3625
Epoch 7/30
250/250 [==============================] - 26s 102ms/step - loss: 2.1502 - accuracy: 0.4124
Epoch 8/30
250/250 [==============================] - 27s 108ms/step - loss: 1.9771 - accuracy: 0.4636
Epoch 9/30
250/250 [==============================] - 25s 100ms/step - loss: 1.8201 - accuracy: 0.5009
Epoch 10/30
250/250 [==============================] - 24s 98ms/step - loss: 1.6610 -

In [8]:
# 모델 저장
from tensorflow.keras.models import save_model

# HDF5 형식으로 저장
save_model(model, 'joint_model.h5')

In [9]:
np.save('fisher_matrix.npy', fisher_matrix)
np.save('optimal_weights.npy', optimal_weights)

c:\ProgramData\Anaconda3\envs\nAIvis\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
